In [1]:
import requests
from bs4 import BeautifulSoup
from unicodedata import normalize
import json
import re
import time

In [49]:
global index
global storaged
global visited_urls
global paginas_descubiertas

visited_urls = set()
paginas_descubiertas = []#clave pagina, valor es la clave
index = 0
storaged = 0

In [15]:
def preproccesText(text):
    # Eliminar tildes de las letras con tilde y mantener la letra ñ intacta
    text = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize("NFD", text), 0, re.I
    )
    text = text.replace("n\u0303", "ñ")  # Restaurar la letra "ñ"

    # Eliminar comandos para fórmulas matemáticas (por ejemplo, {\displaystyle ...})
    text = re.sub(r'{\\[^}]+}', '', text)

    # Eliminar comandos y referencias (por ejemplo, [1], [2], [[Archivo:...])
    text = re.sub(r'\[\d+\]', '', text)  # Eliminar referencias numéricas
    text = re.sub(r'\[\[Archivo:[^\]]+\]\]', '', text)  # Eliminar referencias a archivos
    text = re.sub(r'\[\[[a-zA-Z0-9\s]+\]\]', '', text)  # Eliminar otras referencias en doble corchete

    # Eliminar caracteres no alfabéticos, eliminar saltos de línea adicionales
    text = re.sub(r'(\s*\n\s*)+', ' ', text)
    return text

In [16]:
# Función para extraer el contenido de los párrafos
def extract_content(soup):
    content_data = []
    for paragraph in soup.find_all('p'):
        text = paragraph.get_text()
        text = preproccesText(text)
        content_data.append(text)
    return " ".join(content_data)

In [18]:
def manageLinks(soup):
    # Definir una expresión regular para filtrar los enlaces no deseados
    pattern = re.compile(r"/wiki/.*:|/wiki/Wiki")

    # Extraer los enlaces salientes
    outgoing_links = [a["href"] for a in soup.find_all("a", href=True) if a["href"].startswith("/wiki/") and not pattern.match(a["href"])]
    outgoing_links = [enlace.replace("/wiki/", "") for enlace in outgoing_links]
    return outgoing_links

In [29]:
# Función para realizar web scraping
def scrape_wikipedia(url):
    #start_time = time.time()  # Registrar el tiempo de inicio
    global index
    global visited_urls
    global paginas_descubiertas

    if url in visited_urls:
        #print(f'URL repetida: {url}')
        return

    visited_urls.add(url)
    
    response = requests.get("https://es.wikipedia.org/wiki/" + url)
    if response.status_code == 200:
        page_content = response.text
        soup = BeautifulSoup(page_content, "html.parser")
        
        # Extraer el título de la página
        title = preproccesText(soup.find("h1").text)

        # Extraer el contenido de títulos, subtítulos y párrafos
        content = extract_content(soup)

        # Extraer el snippet (primer párrafo, 200 primeras palabras)
        snippet = content[:200] if content else ""
        
        # Extraer los enlaces salientes
        outgoing_links = manageLinks(soup)
        #los agrega a la lista de enlaces salientes
        paginas_descubiertas.extend(outgoing_links)

        # Imprimir el número de outgoing_links
        #print(f'Página: {url}, outgoing_links: {len(outgoing_links)}')

        # Guardar los datos en un diccionario
        page_data = {
            "url": url,
            "titulo": title,
            "contenido": content,
            "snippet": snippet,
            "enlaces_salientes": outgoing_links
        }

        str = json.dumps(page_data, separators=(',', ':'))
        
        return str

        data.append(page_data)

        for link in outgoing_links:
            next_url = f"https://es.wikipedia.org{link}"
            scrape_wikipedia(next_url, max_pages, current_depth + 1)

    #end_time = time.time()  # Registrar el tiempo de finalización
    #elapsed_time = end_time - start_time
    #print(f'Tiempo para la página : {elapsed_time:.2f} segundos')




In [50]:
paginas_descubiertas.append('Algoritmo')

In [44]:
paginas_descubiertas

['Algoritmo']

In [36]:
# Índices iniciales y finales
tam = 10

real = 0
i = 0


Página: Algoritmo, outgoing_links: 217
URL repetida: Algoritmo
URL repetida: Algoritmo
URL repetida: Algoritmo
Página: Logaritmo, outgoing_links: 240
Página: Diagrama_de_flujo, outgoing_links: 73
Página: Ada_Lovelace, outgoing_links: 143
Página: Matem%C3%A1ticas, outgoing_links: 316
Página: L%C3%B3gica, outgoing_links: 559
Página: Ciencias_de_la_computaci%C3%B3n, outgoing_links: 307
Página: Lat%C3%ADn_tard%C3%ADo, outgoing_links: 94
Página: %C3%81rabe_cl%C3%A1sico, outgoing_links: 34
Página: Empleador, outgoing_links: 37


In [38]:
from tqdm import tqdm 

In [52]:
real = 0
i = 0
tam = 10

In [53]:
real_final = real + tam

# Nombre del archivo de salida
nombre_archivo = f"paginas{real:03d}-{real_final-1:03d}.txt"

# Abre el archivo en modo de escritura
with open(nombre_archivo, "w") as archivo:
    # Utiliza tqdm para crear un indicador de progreso
    with tqdm(total=real_final - real) as pbar:
        while real < real_final:
            if i < len(paginas_descubiertas):
                url = paginas_descubiertas[i]
                resultado = scrape_wikipedia(url)

                if resultado is not None:
                    archivo.write(resultado + "\n")
                    real += 1
                    pbar.update(1)  # Actualiza el indicador de progreso en 1

                i += 1
            else:
                break

 10%|█         | 1/10 [00:00<00:06,  1.31it/s]

Página: Algoritmo, outgoing_links: 217
URL repetida: Algoritmo
URL repetida: Algoritmo
URL repetida: Algoritmo


 20%|██        | 2/10 [00:01<00:06,  1.16it/s]

Página: Logaritmo, outgoing_links: 240


 30%|███       | 3/10 [00:02<00:05,  1.21it/s]

...

Página: Lat%C3%ADn_tard%C3%ADo, outgoing_links: 94


 90%|█████████ | 9/10 [00:09<00:00,  1.01it/s]

Página: %C3%81rabe_cl%C3%A1sico, outgoing_links: 34


100%|██████████| 10/10 [00:09<00:00,  1.02it/s]

Página: Empleador, outgoing_links: 37
